# Imports

In [1]:
import pickle
from preprocess_config import *

# Read + process Data

In [2]:
with open(dataset_dict_path, 'rb') as f:
    raw_dataset_dict = pickle.load(f)

In [3]:
raw_dataset_dict['Admission to movies, theaters, and concerts']

,Category_id,Category,Year,Date,Price,Inflation t-12,Inflation t-11,Inflation t-10,Inflation t-9,Inflation t-8,...,Inflation t-4,Inflation t-3,Inflation t-2,Inflation t-1,Inflation t,Inflation t+1,Indent,Weight,Parent,Parent_ID
1128,290.0,"Admission to movies, theaters, and concerts",2012,2012-01-15,159.002,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-1.013280,7.0,0.331,Admissions,9746.0
1129,290.0,"Admission to movies, theaters, and concerts",2012,2012-02-15,157.399,-1.013280,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.071182,7.0,0.331,Admissions,9746.0
1130,290.0,"Admission to movies, theaters, and concerts",2012,2012-03-15,157.287,-0.071182,-1.013280,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.057839,7.0,0.331,Admissions,9746.0
1131,290.0,"Admission to movies, theaters, and concerts",2012,2012-04-15,157.378,0.057839,-0.071182,-1.013280,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.483649,7.0,0.331,Admissions,9746.0
1132,290.0,"Admission to movies, theaters, and concerts",2012,2012-05-15,158.141,0.483649,0.057839,-0.071182,-1.013280,NaN,...,NaN,NaN,NaN,NaN,NaN,2.390557,7.0,0.331,Admissions,9746.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1264,290.0,"Admission to movies, theaters, and concerts",2023,2023-05-15,217.889,-0.253020,1.411205,1.095213,-0.165095,0.852319,...,-0.583072,-0.571093,1.482856,0.890306,0.010274,0.484850,7.0,0.331,Admissions,9746.0
1265,290.0,"Admission to movies, theaters, and concerts",2023,2023-06-15,218.948,0.484850,-0.253020,1.411205,1.095213,-0.165095,...,0.901122,-0.583072,-0.571093,1.482856,0.890306,0.083547,7.0,0.331,Admissions,9746.0
1266,290.0,"Admission to movies, theaters, and concerts",2023,2023-07-15,219.131,0.083547,0.484850,-0.253020,1.411205,1.095213,...,1.588447,0.901122,-0.583072,-0.571093,1.482856,-0.116893,7.0,0.331,Admissions,9746.0
1267,290.0,"Admission to movies, theaters, and concerts",2023,2023-08-15,218.875,-0.116893,0.083547,0.484850,-0.253020,1.411205,...,-0.263389,1.588447,0.901122,-0.583072,-0.571093,-0.375805,7.0,0.331,Admissions,9746.0


In [5]:
Year

2020

In [6]:
def train_test_split(dic: dict, year: int) -> dict:
    train = {}
    test = {}
    for key in dic.keys():
        category_df = dic[key]
        train[key] = category_df[category_df['Year'] <= year]
        test[key] = category_df[category_df['Year'] > year]
    return train, test

raw_train_data_dict, raw_test_data_dict = train_test_split(raw_dataset_dict, year=Year)

In [7]:
def get_relevant_features(dic: dict, drop_columns: list) -> dict:
    processed_dict = dic.copy()
    for key in list(processed_dict.keys()):
        processed_dict[key] = processed_dict[key].drop(columns=drop_columns)
        processed_dict[key].dropna(inplace=True)
    return processed_dict

drop_columns = ['Category_id', 'Category', 'Year', 'Date', 'Price', 'Indent', 'Weight', 'Parent', 'Parent_ID']

train_data_dict = get_relevant_features(raw_train_data_dict, drop_columns)

test_data_dict = get_relevant_features(raw_test_data_dict, drop_columns)

In [8]:
def get_coefficients(dic: dict) -> dict:
    weight_dict = {}
    for key in list(dic.keys()):
        max_dt = dic[key].Date.max()
        cat_weight = dic[key][dic[key]['Date']==max_dt].Weight.values[0]
        weight_dict[key] = cat_weight
    return weight_dict

weight_dict = get_coefficients(raw_train_data_dict)

In [9]:
def get_sons(dic: dict) -> dict:
    for key in list(dic.keys()):
        max_dt = dic[key].Date.max()
        cat_weight = dic[key][dic[key]['Date']==max_dt].Weight.values[0]
        weight_dict[key] = cat_weight
    return weight_dict

In [10]:
train_data_dict['Rent of shelter']

,Inflation t-12,Inflation t-11,Inflation t-10,Inflation t-9,Inflation t-8,Inflation t-7,Inflation t-6,Inflation t-5,Inflation t-4,Inflation t-3,Inflation t-2,Inflation t-1,Inflation t,Inflation t+1
57336,0.294378,0.769462,-0.178094,0.000000,0.237530,0.000000,0.536514,0.419288,0.360794,-0.060223,-0.120337,0.421814,0.666467,0.352113
57337,0.352113,0.294378,0.769462,-0.178094,0.000000,0.237530,0.000000,0.536514,0.419288,0.360794,-0.060223,-0.120337,0.421814,0.350878
57338,0.350878,0.352113,0.294378,0.769462,-0.178094,0.000000,0.237530,0.000000,0.536514,0.419288,0.360794,-0.060223,-0.120337,0.291460
57339,0.291460,0.350878,0.352113,0.294378,0.769462,-0.178094,0.000000,0.237530,0.000000,0.536514,0.419288,0.360794,-0.060223,0.348635
57340,0.348635,0.291460,0.350878,0.352113,0.294378,0.769462,-0.178094,0.000000,0.237530,0.000000,0.536514,0.419288,0.360794,0.231750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57642,0.129478,0.243262,0.120526,0.216664,-0.036931,0.092501,0.315975,0.382457,0.215199,0.261723,0.032015,0.318918,0.148115,0.079483
57643,0.079483,0.129478,0.243262,0.120526,0.216664,-0.036931,0.092501,0.315975,0.382457,0.215199,0.261723,0.032015,0.318918,0.048949
57644,0.048949,0.079483,0.129478,0.243262,0.120526,0.216664,-0.036931,0.092501,0.315975,0.382457,0.215199,0.261723,0.032015,0.162795
57645,0.162795,0.048949,0.079483,0.129478,0.243262,0.120526,0.216664,-0.036931,0.092501,0.315975,0.382457,0.215199,0.261723,0.120462


In [11]:
with open(train_dataset_path, 'wb') as handle:
    pickle.dump(train_data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(test_dataset_path, 'wb') as handle:
    pickle.dump(test_data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(coefficient_path, 'wb') as handle:
    pickle.dump(weight_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

-------

# Horizon Test Sets:

In [12]:
def get_relevant_features_horizon(dic: dict, drop_columns: list) -> dict:
    processed_dict = dic.copy()
    for key in list(processed_dict.keys()):
        processed_dict[key] = processed_dict[key].drop(columns=drop_columns)
        #processed_dict[key].dropna(inplace=True)
    return processed_dict

drop_columns = ['Category_id', 'Category', 'Year', 'Date', 'Price', 'Indent', 'Weight', 'Parent', 'Parent_ID']

In [13]:
with open(dataset_dict_path, 'rb') as f:
    hor_1_dataset_dict = pickle.load(f)

hor1_train_data_dict, hor1_test_data_dict = train_test_split(hor_1_dataset_dict, year=Year)
hor1_test_data_dict = get_relevant_features_horizon(hor1_test_data_dict, drop_columns)

print(f'columns: {hor1_test_data_dict["All items"].columns}')
print(f'shape: {hor1_test_data_dict["All items"].shape}')


columns: Index(['Inflation t-12', 'Inflation t-11', 'Inflation t-10', 'Inflation t-9',
       'Inflation t-8', 'Inflation t-7', 'Inflation t-6', 'Inflation t-5',
       'Inflation t-4', 'Inflation t-3', 'Inflation t-2', 'Inflation t-1',
       'Inflation t', 'Inflation t+1'],
      dtype='object')
shape: (33, 14)


In [14]:
with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/bi_directional_2_period_dataset_dict.pickle', 'rb') as f:
    hor_2_dataset_dict = pickle.load(f)

hor2_train_data_dict, hor2_test_data_dict = train_test_split(hor_2_dataset_dict, year=Year)
hor2_test_data_dict = get_relevant_features_horizon(hor2_test_data_dict, drop_columns)

print(f'columns: {hor2_test_data_dict["All items"].columns}')
print(f'shape: {hor2_test_data_dict["All items"].shape}')


columns: Index(['Inflation t-12', 'Inflation t-11', 'Inflation t-10', 'Inflation t-9',
       'Inflation t-8', 'Inflation t-7', 'Inflation t-6', 'Inflation t-5',
       'Inflation t-4', 'Inflation t-3', 'Inflation t-2', 'Inflation t-1',
       'Inflation t', 'Inflation t+1', 'Inflation t+2'],
      dtype='object')
shape: (33, 15)


In [15]:
hor2_test_data_dict['All items'].shape

(33, 15)

In [16]:
with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/bi_directional_3_period_dataset_dict.pickle', 'rb') as f:
    hor_3_dataset_dict = pickle.load(f)

hor3_train_data_dict, hor3_test_data_dict = train_test_split(hor_3_dataset_dict, year=Year)
hor3_test_data_dict = get_relevant_features_horizon(hor3_test_data_dict, drop_columns)

print(f'columns: {hor3_test_data_dict["All items"].columns}')
print(f'shape: {hor3_test_data_dict["All items"].shape}')


columns: Index(['Inflation t-12', 'Inflation t-11', 'Inflation t-10', 'Inflation t-9',
       'Inflation t-8', 'Inflation t-7', 'Inflation t-6', 'Inflation t-5',
       'Inflation t-4', 'Inflation t-3', 'Inflation t-2', 'Inflation t-1',
       'Inflation t', 'Inflation t+1', 'Inflation t+2', 'Inflation t+3'],
      dtype='object')
shape: (33, 16)


In [17]:
with open('//Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/bi_directional_4_period_dataset_dict.pickle', 'rb') as f:
    hor_4_dataset_dict = pickle.load(f)

hor4_train_data_dict, hor4_test_data_dict = train_test_split(hor_4_dataset_dict, year=Year)
hor4_test_data_dict = get_relevant_features_horizon(hor4_test_data_dict, drop_columns)


print(f'columns: {hor4_test_data_dict["All items"].columns}')
print(f'shape: {hor4_test_data_dict["All items"].shape}')


columns: Index(['Inflation t-12', 'Inflation t-11', 'Inflation t-10', 'Inflation t-9',
       'Inflation t-8', 'Inflation t-7', 'Inflation t-6', 'Inflation t-5',
       'Inflation t-4', 'Inflation t-3', 'Inflation t-2', 'Inflation t-1',
       'Inflation t', 'Inflation t+1', 'Inflation t+2', 'Inflation t+3',
       'Inflation t+4'],
      dtype='object')
shape: (33, 17)


In [18]:
with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/bi_directional_8_period_dataset_dict.pickle', 'rb') as f:
    hor_8_dataset_dict = pickle.load(f)

hor8_train_data_dict, hor8_test_data_dict = train_test_split(hor_8_dataset_dict, year=Year)
hor8_test_data_dict = get_relevant_features_horizon(hor8_test_data_dict, drop_columns)


print(f'columns: {hor8_test_data_dict["All items"].columns}')
print(f'shape: {hor8_test_data_dict["All items"].shape}')


columns: Index(['Inflation t-12', 'Inflation t-11', 'Inflation t-10', 'Inflation t-9',
       'Inflation t-8', 'Inflation t-7', 'Inflation t-6', 'Inflation t-5',
       'Inflation t-4', 'Inflation t-3', 'Inflation t-2', 'Inflation t-1',
       'Inflation t', 'Inflation t+1', 'Inflation t+2', 'Inflation t+3',
       'Inflation t+4', 'Inflation t+5', 'Inflation t+6', 'Inflation t+7',
       'Inflation t+8'],
      dtype='object')
shape: (33, 21)


In [19]:
with open(hor1_test_dataset_path, 'wb') as handle:
    pickle.dump(hor1_test_data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(hor2_test_dataset_path, 'wb') as handle:
    pickle.dump(hor2_test_data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(hor3_test_dataset_path, 'wb') as handle:
    pickle.dump(hor3_test_data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(hor4_test_dataset_path, 'wb') as handle:
    pickle.dump(hor4_test_data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(hor8_test_dataset_path, 'wb') as handle:
    pickle.dump(hor8_test_data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)